In [ ]:
# http://stackoverflow.com/questions/44544766/ddg#44547144
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import numpy as np
import pandas as pd
import time
import seaborn as sns
from matplotlib import pyplot as plt
import keras

In [ ]:
def pd_read_csv(fn:str):
    x = pd.read_csv(fn, na_values=['NO_LABEL', '(blank)'])
    assert not x['Unnamed: 0'].duplicated().any()
    x = x.set_index("Unnamed: 0")
    return x

## Read data

In [ ]:
train = pd_read_csv('data_in/TrainingData.csv')
test = pd_read_csv('data_in/TestData.csv')

In [ ]:
# train.columns
train['Position_Type'].head()

In [ ]:
features = list(set(train.columns).intersection(set(test.columns)) - set(['FTE','Total']))
features.sort()
features

In [ ]:
target = set(train.columns) - set(test.columns)
target = list(target)
target.sort()
target

In [ ]:
for col in target:
    test[col] = np.nan

In [ ]:
train.shape, test.shape

In [ ]:
train['is_holdout'] = False
test ['is_holdout'] = True
df = pd.concat([train,test], axis=0)
df.shape

In [ ]:
# Analyse FTE for classification

# df['FTE'].value_counts().iloc[:60].sum(), df['FTE'].value_counts().iloc[60:].sum(), pd.isnull(df['FTE']).sum()
# df['FTE'].value_counts().head(n=40).sum()

# df['FTE2'] = df['FTE'].round(1)

df['FTE'].apply(lambda x: 'nan' if pd.isnull(x) else ( str(round(x,1)) if x <=1 else '>1' ) ).value_counts() # .shape # .tail()

In [ ]:
# convert FTE to string boolean and append to list of features
# df['FTE'] = ~pd.isnull(df['FTE']).astype('str')
df['FTE'] = df['FTE'].apply(lambda x: 'nan' if pd.isnull(x) else ( str(round(x,1)) if x <=1 else '>1' ) )
features = features + ['FTE']
features.sort()
features

In [ ]:
meta = list(set(df.columns) - set(features) - set(target))
meta

In [ ]:
df.shape, df[features].shape, df[target].shape, df[meta].shape

## Analyze how close the train and test features are

In [ ]:
results = []
for ff in features:
    vc_train = train[ff].value_counts()
    vc_test  = test [ff].value_counts()
    # vc_train.shape, vc_test.shape
    vc_both  = vc_train.reset_index().merge(
        vc_test.reset_index(), 
        left_on = 'index', 
        right_on='index', 
        how='outer', 
        suffixes=['_train', '_test']
    )
    vc_both = vc_both.set_index('index')
    # vc_both.head()
    # vc_both[pd.isnull(vc_both['Facility_or_Department_test'])].head()
    out = {
        'feature': ff,
        'train all': train.shape[0],
        # 'train': vc_both['%s_train'%ff].sum(),
        'train non-null': (~pd.isnull(train[ff])).sum(),
        'train_minus_test': vc_both['%s_train'%ff][pd.isnull(vc_both['%s_test'%ff ])].sum(), 
        'test_minus_train': vc_both['%s_test'%ff ][pd.isnull(vc_both['%s_train'%ff])].sum(),
    }
    out['tmt_pct'] = out['test_minus_train'] * 100 // out['train non-null']
    results.append(out)


results = pd.DataFrame(results)
results = results.set_index('feature').sort_index()
results = results.astype('uint32')

In [ ]:
# results.shape
# results.head()
results[['train all', 'train non-null', 'train_minus_test', 'test_minus_train', 'tmt_pct']]

In [ ]:
# sod = train['Sub_Object_Description'].value_counts()
sod = test['Sub_Object_Description'].value_counts()

In [ ]:
sod.head(n=20)

In [ ]:
from matplotlib import pyplot as plt
# plt.bar(x=range(sod.shape[0]), height=sod.values)
plt.bar(x=range(sod.shape[0]-5), height=sod.iloc[5:].values)
plt.show()

In [ ]:
# sod[sod<10].shape[0], sod.shape[0]
sod[sod<10]

In [ ]:
subtest = test['Sub_Object_Description'].apply(lambda x: (~pd.isnull(x)) & ('community' in str(x).lower())) # .sum()

In [ ]:
test['Sub_Object_Description'][subtest].head()

## Read target labels

In [ ]:
import yaml
labels = yaml.load(open("labels.yml",'r'))

In [ ]:
# Function': ['Aides Compensation
prediction_names = []
for k,v1 in labels.items():
    for v2 in v1:
        pn = "%s__%s"%(k,v2)
        prediction_names.append(pn)
        
        
assert 'Function__Aides Compensation' in prediction_names
prediction_names.sort()
prediction_names[:5]

## one-hot encode each target by its classes

In [ ]:
for p in prediction_names: df[p] = False

In [ ]:
for k,v1 in labels.items():
    for v2 in v1:
        pn = "%s__%s"%(k,v2)
        # print(pn)
        df[pn] = df[k] == v2

In [ ]:
# since NO_LABEL is replaced with NaN, need this
for dependent in labels.keys():
    target_sub = [x for x in df.columns if x.startswith("%s__"%dependent)]
    df.loc[~df[target_sub].any(axis=1), '%s__NO_LABEL'%dependent]=True

In [ ]:
df[['Function', 'Function__Teacher Compensation', 'Function__Substitute Compensation', 'Function__NO_LABEL']].head()

In [ ]:
df.shape, df[pd.isnull(df[prediction_names]).all(axis=1)].shape, df.loc[~df[prediction_names].any(axis=1)].shape

In [ ]:
assert ~pd.isnull(df[prediction_names]).any().any()

In [ ]:
df[prediction_names] = df[prediction_names].astype('uint8')

## Factorize features

In [ ]:
print(time.ctime())
df_feat = df[features].apply(lambda x: pd.factorize(x)[0], axis=0)
df_feat = df_feat + 1 # +1 for the -1 from pd.factorize on nan (keras Embedding supports [0,N) )    
print(time.ctime())

In [ ]:
df_feat.max().max(), df_feat.min().min()

In [ ]:
vocab_size = df_feat.max(axis=0) + 1 # +1 to count the 0 index
vocab_size = vocab_size.sort_index()
vocab_size

In [ ]:
assert df[prediction_names].max().max()==1

## split the non-holdout into train/test

In [ ]:
x = df_feat[~df['is_holdout']]
y = df[prediction_names][~df['is_holdout']] # .fillna(0)

from sklearn.model_selection import train_test_split
# test_size=0.33
test_size=0
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=42)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
# calculate label_keys array whose order is replicable
label_keys = labels.keys()
label_keys = list(label_keys)
label_keys.sort()


## build a dummy equi-probable target

In [ ]:
y_equi = {}
for k in label_keys:
    y_equi[k] = np.ones(shape=(y_train.shape[0], len(labels[k]))) / len(labels[k])

y_equi = [y_equi[k] for k in label_keys]
y_equi = np.concatenate(y_equi, axis=1)
y_equi = pd.DataFrame(y_equi, columns=y_train.columns, index=y_train.index)
y_equi.shape

In [ ]:
y_equi.head()
# y_equi[0].head()

## keras embedding + Dense/LSTM

In [ ]:
from keras.layers import Embedding, Dense, Flatten, LSTM, Input, Concatenate, Add, Lambda, Dropout
from keras.models import Sequential, Model
from keras import backend as K

def build_fn():

    # vocab_size = stats.shape[0]

    # inputs = [Input(shape=(prob3.shape[1],)) for f in vocab_size.index]
    inputs = {f: Input(shape=(1,), name=f) for f in vocab_size.index}

    # embeddings = [Embedding(vocab_size[f], embedding_dim, input_length=prob3.shape[1]) for f in vocab_size.index]

    if True:
        embedding_dim = 10 # 3 # 12 # 2 # 64 # FIXME
        embeddings = {f: Embedding(vocab_size[f], embedding_dim, input_length=1)(inputs[f]) for f in vocab_size.index}
    else:
        embeddings = {f: Embedding(vocab_size[f], max(3, vocab_size[f]//15//10), input_length=1)(inputs[f]) for f in vocab_size.index}

    # the model will take as input an integer matrix of size (batch, input_length).
    # the largest integer (i.e. word index) in the input should be no larger than 999 (vocabulary size).
    # now model.output_shape == (None, input_length, embedding_dim), where None is the batch dimension.

    # dummy variable
    x1= embeddings

    #  flatten each feature since no sequences anyway
    x1 = {f: Flatten(name="%s_flat"%f)(x1[f]) for f in vocab_size.index}

    # dense layer for each feature
    # x1 = {f: Dense(10, activation = 'relu', name="%s_d01"%f)(x1[f]) for f in vocab_size.index}
    # x1 = {f: Dense( 3, activation = 'relu', name="%s_d02"%f)(x1[f]) for f in vocab_size.index}

    # a dropout for each feature, this way, the network is more robust to dependencies on a single feature
    x1 = {f: Dropout(0.3, name="%s_dropout"%f)(x1[f]) for f in vocab_size.index}

    x1 = [x1[f] for f in vocab_size.index]
    x1 = Concatenate()(x1)
    # x1 = Flatten()(x1)
    x1 = Dropout(0.3)(x1)

    x1 = Dense(1000, activation='relu')(x1)
    x1 = Dense( 300, activation='relu')(x1)

    # x1 = Dense( 50, activation='relu')(x1)
    # o1 = {dependent: Dense(50, activation = 'relu', name="%s_d1"%dependent)(x1) for dependent in label_keys}
    # o1 = {dependent: Dense(50, activation = 'relu', name="%s_d2"%dependent)(o1[dependent]) for dependent in label_keys}

    # outputs = [Dense(len(labels[dependent]), activation = 'softmax', name="%s_out"%dependent)(o1[dependent]) for dependent in label_keys]
    outputs = [Dense(len(labels[dependent]), activation = 'softmax', name="%s_out"%dependent)(x1) for dependent in label_keys]

    inputs = [inputs[f] for f in vocab_size.index]
    model = Model(inputs=inputs, outputs=outputs)
    model.summary()
    
    # model.compile('rmsprop', loss=multi_multiclass_logloss, metrics=['acc'])
    # model.compile('rmsprop', loss='categorical_crossentropy', metrics=['acc'])
    model.compile('adam', loss='categorical_crossentropy', metrics=['acc'])
    
    return model

In [ ]:
from keras.wrappers.scikit_learn import BaseWrapper

class BaseWrapper2(BaseWrapper):
    """
    Mostly just a copy of the original class in order to introduce preprocess_feature and preprocess_target
    """
    def fit(self, x, y, **kwargs):
        if self.build_fn is None:
            self.model = self.__call__(**self.filter_sk_params(self.__call__))
        elif (not isinstance(self.build_fn, types.FunctionType) and
              not isinstance(self.build_fn, types.MethodType)):
            self.model = self.build_fn(
                **self.filter_sk_params(self.build_fn.__call__))
        else:
            self.model = self.build_fn(**self.filter_sk_params(self.build_fn))

        loss_name = self.model.loss
        if hasattr(loss_name, '__name__'):
            loss_name = loss_name.__name__
        if loss_name == 'categorical_crossentropy' and len(y.shape) != 2:
            y = to_categorical(y)

        fit_args = copy.deepcopy(self.filter_sk_params(Sequential.fit))
        fit_args.update(kwargs)
        
        # new lines
        x = self.preprocess_feature(x) # <<<<<<<
        y = self.preprocess_target(y) # <<<<<<<

        history = self.model.fit(x, y, **fit_args)

        return history

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier

class KerasClassifier2(KerasClassifier):
    def fit(self, x, y, **kwargs):
        y = np.array(y)
        if len(y.shape) == 2 and y.shape[1] > 1:
            self.classes_ = np.arange(y.shape[1])
        elif (len(y.shape) == 2 and y.shape[1] == 1) or len(y.shape) == 1:
            self.classes_ = np.unique(y)
            y = np.searchsorted(self.classes_, y)
        else:
            raise ValueError('Invalid shape for y: ' + str(y.shape))
        self.n_classes_ = len(self.classes_)
        return super(KerasClassifier, self).fit(x, y, **kwargs)


In [ ]:
class MyNetwork(KerasClassifier):
    def preprocess_feature(self, x):
        """
        convert 2-D matrix of features into array of 1-D features
        This is needed because each feature has a different vocabulary for its embedding
        """
        x = [x[f].values for f in vocab_size.index]
        return x
    
    def preprocess_target(self, y):
        """
        convert 2-D matrix of targets into K arrays of C-D matrices 
        where C is the number of classes of each target
        """
        y = [y[[x for x in prediction_names if x.startswith("%s__"%f)]].values for f in label_keys]
        return y
    
    def fit(self, x, y, **kwargs):
        x = self.preprocess_feature(x)
        y = self.preprocess_target(y)
        super().fit(x,y,**kwargs)
        
    def predict(self, x, **kwargs):
        x = self.preprocess_feature(x)
        super().predict(x,**kwargs)
        
    def predict_proba(self, x, **kwargs):
        x = self.preprocess_feature(x)
        super().predict_proba(x,**kwargs)
        
    def score(self, x, y, **kwargs):
        x = self.preprocess_feature(x)
        super().score(x,**kwargs)
        
    def __call__():
        build_fn()

In [ ]:
keras_clf = MyNetwork(build_fn=None, epochs=30, batch_size=32*32, verbose=0)

# from sklearn.cross_validation import StratifiedKFold
# skf = cross_validation.StratifiedKFold(y_train, n_folds=3, shuffle=True)

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3, shuffle=True)

from sklearn.calibration import CalibratedClassifierCV
model = CalibratedClassifierCV(keras_clf, cv=skf, method='sigmoid')

In [ ]:
keras_clf.fit(x_train, y_train)

In [ ]:
# then train to actual probabilities
history = model.fit(x_train, y_train)

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()

In [ ]:
model.evaluate(x_test, y_test, batch_size = 32*32)

## argmax accuracy

## Spatial comparison

In [ ]:
# sub_labels = labels
sub_labels = {k:labels[k] for k in label_keys if k in ['Function']}

n_show = 1000
y_pred = model.predict(x_test)

In [ ]:
for i,v0 in enumerate(sub_labels.items()):
    k,v1 = v0
    y_pred2 = pd.DataFrame(y_pred[i], columns=v1)
    y_test2 = pd.DataFrame(y_test[i], columns=v1)
    for v2 in v1:
        plt.figure(figsize=(20,3))
        plt.plot(y_pred2.loc[:n_show,v2], label='pred')
        #plt.plot(sum_pred, label='sum_pred', alpha=0.2)
        plt.plot(y_test2.loc[:n_show,v2], '.', label='actual')
        plt.legend(loc='best')
        plt.title("%s: %s"%(k,v2))

        axes = plt.gca()
        axes.set_ylim([-.1,1.1])
        plt.show()

In [ ]:
y_test[0][0,:].sum(), y_pred[0][0,:].sum() # , y_pred[0]

## temporal comparison

In [ ]:
y_pred = model.predict(x_test)
y_pred2 = {label_keys[i]: y_pred[i] for i in range(len(label_keys))}
y_test2 = {label_keys[i]: y_test[i] for i in range(len(label_keys))}

k2 = 'Function'
y_pred3 = y_pred2[k2]
y_test3 = y_test2[k2]

for i in range(15):
    plt.figure(figsize=(10,3))
    
    plt.subplot(121)
    plt.bar(x=range(y_pred3.shape[1]), height=y_test3[i])
    plt.title('%s. actual, argmax=%s'%(i,np.argmax(y_test3[i])))
    axes = plt.gca()
    axes.set_ylim([-.1,1.1])
    
    plt.subplot(122)
    plt.bar(x=range(y_pred3.shape[1]), height=y_pred3[i])
    plt.title('%s. prediction, argmax=%s'%(i,np.argmax(y_pred3[i])))
    axes = plt.gca()
    axes.set_ylim([-.1,1.1])
    
    # plt.title(y_test.index[i])
    
    plt.show()

In [ ]:
i=6
y_test3[i,:]

## Mock submission

In [ ]:
x_ho = df_feat[features][~df['is_holdout']].head()
x_ho  = [x_ho [f].values for f in vocab_size.index]
y_ho = model.predict(x_ho)
df_submit = pd.DataFrame(np.concatenate(y_ho, axis=1), columns=prediction_names)
df_submit.shape

In [ ]:
df_submit.head()

In [ ]:
df[target].head()

## Prepare submission

In [ ]:
df.shape, df_feat.shape

In [ ]:
x_ho = df_feat[features][ df['is_holdout']]
x_ho  = [x_ho [f].values for f in vocab_size.index]
y_ho = model.predict(x_ho)

In [ ]:
len(y_ho), y_ho[0].shape, y_ho[1].shape

In [ ]:
df_submit = pd.DataFrame(np.concatenate(y_ho, axis=1), columns=prediction_names, index=df_feat[ df['is_holdout']].index)
df_submit.shape

In [ ]:
df_submit.head()

In [ ]:
# plt.plot(df_submit['Use__NO_LABEL'].sort_values().values)
plt.plot(df_submit['Operating_Status__NO_LABEL'].sort_values().values)
plt.show()

In [ ]:
test.head()

In [ ]:
assert (df_submit['Operating_Status__NO_LABEL']<0.0001).all()
del df_submit['Operating_Status__NO_LABEL']

In [ ]:
fn = 'data_out/submission_C4_%s.csv'%(time.strftime("%Y%m%d_%H%M%S"))
df_submit.to_csv(fn)

## compare to C3

In [ ]:
fn = 'data_out/submission_C3_20180409_120904.csv'
df_c3 = pd.read_csv(fn).set_index("Unnamed: 0")

In [ ]:
(df_submit - df_c3).head(n=2).apply(lambda x: x.idxmax(), axis=1)

In [ ]:
df.loc[[180042,28872], ['FTE']].head(n=2)

In [ ]:
result = (df_submit - df_c3).max(axis=1)

In [ ]:
result.shape[0], result[abs(result) > 0.4].shape[0]

In [ ]:
result = df_submit.apply(lambda x: x.idxmax(), axis=1) != df_c3.apply(lambda x: x.idxmax(), axis=1)

In [ ]:
result.value_counts()

## RF